# Basic CNN model

## Import libraries

In [1]:
%matplotlib inline
import numpy as np
from utils.helper import read_data
import utils.processing as processing
import matplotlib.pyplot as plt
FOLDER = "data/"
SAMPLE_SIZE = 1000
N_STATIC = 3
N_MOVING = 3

## Data preprocessing

Here we are reading the data from the files and combining the phases of size **SAMPLE_SIZE**. 

In [2]:
data_static = [read_data(FOLDER + "data_static_100_s_" + str(i+1) + ".csv") for i in range(N_STATIC)]
data_moving = [read_data(FOLDER + "data_moving_100_s_" + str(i+1) + ".csv") for i in range(N_MOVING)]
print(f"Shape one of three static datas: {data_static[0].shape}")
print(f"Shape one of three moving datas: {data_moving[0].shape}")

Shape one of three static datas: (3, 100000, 64)
Shape one of three moving datas: (3, 100000, 64)


In [3]:
from utils.helper import calculate_phases
phases_static = calculate_phases(data_static, N_STATIC, SAMPLE_SIZE)
phases_moving = calculate_phases(data_moving, N_STATIC, SAMPLE_SIZE)
print(f"Shape static phases: {phases_static.shape}")
print(f"Shape moving phases: {phases_moving.shape}")

train_data = np.concatenate((phases_static, phases_moving))
print(f"Shape train data: {train_data.shape}")

Shape static phases: (300, 1000)
Shape moving phases: (300, 1000)
Shape train data: (600, 1000)


Now we need to create labels for the training data.

In [4]:
indexes = np.arange(train_data.shape[0])
train_labels = indexes >= train_data.shape[0]/2
print("Shape train labels:", train_labels.shape)

Shape train labels: (600,)


## Model

### Simple Threshold classifier

In [5]:
from sklearn.metrics import classification_report
means = np.mean(train_data, axis=1)
classified_data = means > 30 #False = not moving True = moving

print(classification_report(train_labels, classified_data))

              precision    recall  f1-score   support

       False       0.65      1.00      0.79       300
        True       1.00      0.46      0.63       300

    accuracy                           0.73       600
   macro avg       0.82      0.73      0.71       600
weighted avg       0.82      0.73      0.71       600

